In [120]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm
import numpy as np
import pandas as pd
torch.manual_seed(42)

In [121]:
import string
string_arr =[]
pos_arr=[]
actions_arr=[]
with open("train.txt") as fic:
    for line in fic:
      lines = line.rstrip('\n').split('|||')
      line0 = lines[0].split(' ')
      line1 = lines[1].split(' ')
      line2 = lines[2].split(' ')
      string_arr.append(line0[0:-1])
      pos_arr.append(line1[1:-1])
      actions_arr.append(line2[1:])

In [122]:
# actions_arr = actions_arr[0:10]
# string_arr = string_arr[0:10]

In [5]:
import torchtext.vocab as vocab
examples = string_arr[0]
vec = vocab.GloVe(name='6B', dim=50)
ret = vec.get_vecs_by_tokens(examples, lower_case_backup=True)
# print(ret)

.vector_cache/glove.6B.zip: 862MB [02:39, 5.41MB/s]                           
100%|█████████▉| 399999/400000 [00:15<00:00, 25800.86it/s]


In [123]:
from typing import List, Set

class Token:
    def __init__(self, idx: int, word: str, pos: str):
        self.idx = idx # Unique index of the token
        self.word = word # Token string
        self.pos  = pos # Part of speech tag

class DependencyEdge:
    def __init__(self, source: Token, target: Token, label:str):
        self.source = source  # Source token index
        self.target = target  # target token index
        self.label  = label  # dependency label
        pass


class ParseState:
    def __init__(self, stack: List[Token], parse_buffer: List[Token], dependencies: List[DependencyEdge]):
        self.stack = stack # A stack of token indices in the sentence. Assumption: the root token has index 0, the rest of the tokens in the sentence starts with 1.
        self.parse_buffer = parse_buffer  # A buffer of token indices
        self.dependencies = dependencies
        pass

    def add_dependency(self, source_token, target_token, label):
        self.dependencies.append(
            DependencyEdge(
                source=source_token,
                target=target_token,
                label=label,
            )
        )


def shift(state: ParseState) -> None:
    stack1 = state.stack
    parse_buffer1 = state.parse_buffer
    dependencies1 = state.dependencies
    temp = parse_buffer1.pop(0)
    stack1.append(temp)
    # TODO: Implement this as an in-place operation that updates the parse state and does not return anything

    # The python documentation has some pointers on how lists can be used as stacks and queues. This may come useful:
    # https://docs.python.org/3/tutorial/datastructures.html#using-lists-as-stacks
    # https://docs.python.org/3/tutorial/datastructures.html#using-lists-as-queues

    pass


def left_arc(state: ParseState, label: str) -> None:
    stack1 = state.stack
    parse_buffer1 = state.parse_buffer
    dependencies1 = state.dependencies
    temp1 = stack1.pop()
    temp2 = stack1.pop()
    stack1.append(temp1)
    state.add_dependency(temp1,temp2,label)
    # TODO: Implement this as an in-place operation that updates the parse state and does not return anything

    # The python documentation has some pointers on how lists can be used as stacks and queues. This may come useful:
    # https://docs.python.org/3/tutorial/datastructures.html#using-lists-as-stacks
    # https://docs.python.org/3/tutorial/datastructures.html#using-lists-as-queues

    # Also, you will need to use the state.add_dependency method defined above.

    pass


def right_arc(state: ParseState, label: str) -> None:
    stack1 = state.stack
    parse_buffer1 = state.parse_buffer
    dependencies1 = state.dependencies
    temp1 = stack1.pop()
    temp2 = stack1.pop()
    stack1.append(temp2)
    state.add_dependency(temp2,temp1,label)
    # TODO: Implement this as an in-place operation that updates the parse state and does not return anything

    # The python documentation has some pointers on how lists can be used as stacks and queues. This may come useful:
    # https://docs.python.org/3/tutorial/datastructures.html#using-lists-as-stacks
    # https://docs.python.org/3/tutorial/datastructures.html#using-lists-as-queues

    # Also, you will need to use the state.add_dependency method defined above.

    pass



def is_final_state(state: ParseState,cwindow: int) -> bool:
    # TODO: Implemement this
    stack1 = state.stack
    parse_buffer1 = state.parse_buffer
    dependencies1 = state.dependencies

    if(len(parse_buffer1) <= 2) and len(stack1) <= 3:
      return True
    else:
      # print( len(stack1),len(parse_buffer1))
      return False


def find_action(state: ParseState,out: List[float]) -> str:
    stack1 = state.stack
    parse_buffer1 = state.parse_buffer
    dependencies1 = state.dependencies
    out_put = out
    max_index = 0
    max_0_index = 1
    for i in range(len(out_put)):
      if(i>0):
        if out_put[max_0_index] < out_put[i]:
          max_0_index = i
      if(i>=0):
        if out_put[max_index] < out_put[i]:
          max_index = i
    if len(stack1) <= 3:
      return rev_tags_dict[0]
    elif len(parse_buffer1) <= 2:
      return rev_tags_dict[max_0_index]
    else:
      return rev_tags_dict[max_index]


def get_deps(words_lists, actions, cwindow):
    """ Computes all the dependencies set for all the sentences according to
    actions provided
    Inputs
    -----------
    words_lists: List[List[str]].  This is a list of lists. Each inner list is a list of words in a sentence,
    actions: List[List[str]]. This is a list of lists where each inner list is the sequence of actions
                Note that the elements should be valid actions as in `tagset.txt`
    cwindow: int. Context window. Default=2
    """
    all_deps = []   # List of List of dependencies
    # Iterate over sentences
    for w_ix, words_list in enumerate(words_lists):
        # Intialize stack and buffer appropriately
        stack = [Token(idx=-i-1, word="[NULL]", pos="NULL") for i in range(cwindow)]
        parser_buff = []
        for ix in range(len(words_list)):
            parser_buff.append(Token(idx=ix, word=words_list[ix], pos="NULL"))
        parser_buff.extend([Token(idx=ix+i+1, word="[NULL]",pos="NULL") for i in range(cwindow)])
        # Initilaze the parse state
        state = ParseState(stack=stack, parse_buffer=parser_buff, dependencies=[])

        # Iterate over the actions and do the necessary state changes
        for action in actions[w_ix]:
            if action == "SHIFT":
                shift(state)
            elif action[:8] == "REDUCE_L":
                left_arc(state, action[9:])
            else:
                right_arc(state, action[9:])
        assert is_final_state(state,cwindow)    # Check to see that the parse is complete
        right_arc(state, "root")    # Add te root dependency for the remaining element on stack
        all_deps.append(state.dependencies.copy())  # Copy over the dependenices found
    return all_deps




def compute_metrics(words_lists, gold_actions, pred_actions, cwindow=2):
    """ Computes the UAS and LAS metrics given list of words, gold and predicted actions.
    Inputs
    -------
    word_lists: List[List[str]]. This is a list of lists. Each inner list is a list of words in a sentence,
    gold_action: List[List[str]]. This is a list of lists where each inner list is the sequence of gold actions
                Note that the elements should be valid actions as in `tagset.txt`
    pred_action: List[List[str]]. This is a list of lists where each inner list is the sequence of predicted actions
                Note that the elements should be valid actions as in `tagset.txt`

    Outputs
    -------
    uas: int. The Unlabeled Attachment Score
    las: int. The Lableled Attachment Score
    """
    lab_match = 0  # Counter for computing correct head assignment and dep label
    unlab_match = 0 # Counter for computing correct head assignments
    total = 0       # Total tokens

    # Get all the dependencies for all the sentences
    gold_deps = get_deps(words_lists, gold_actions, cwindow)    # Dep according to gold actions
    pred_deps = get_deps(words_lists, pred_actions, cwindow)    # Dep according to predicted actions

    # Iterate over sentences
    for w_ix, words_list in enumerate(words_lists):
        # Iterate over words in a sentence
        for ix, word in enumerate(words_list):
            # Check what is the head of the word in the gold dependencies and its label
            for dep in gold_deps[w_ix]:
                if dep.target.idx == ix:
                    gold_head_ix = dep.source.idx
                    gold_label = dep.label
                    break
            # Check what is the head of the word in the predicted dependencies and its label
            for dep in pred_deps[w_ix]:
                if dep.target.idx == ix:
                    # Do the gold and predicted head match?
                    if dep.source.idx == gold_head_ix:
                        unlab_match += 1
                        # Does the label match?
                        if dep.label == gold_label:
                            lab_match += 1
                    break
            total += 1

    return unlab_match/total, lab_match/total

def find_left(word,dep,left_list):
  if(len(left_list)==0):
    return 'NULL'
  temp = word
  while(temp in left_list):
    for edge in dep:
      if edge.source == temp:
        temp = edge.target
  if(temp == word):
    return 'NULL'
  else:
    return temp

def find_right(word,dep,right_list):
  if(len(right_list)==0):
    return 'NULL'
  temp = word
  while(temp in right_list):
    for edge in dep:
      if edge.source == temp:
        temp = edge.target
  if(temp == word):
    return 'NULL'
  else:
    return temp


def left_right(state: ParseState):
    stack1 = state.stack
    parse_buffer1 = state.parse_buffer
    dependencies1 = state.dependencies
    left_list = []
    right_list = []
    for edge in dependencies1:
      if tags_dict[edge.label]%2 == 1:
        left_list.append(edge.source)
      else:
        right_list.append(edge.source)
    st1 = stack1[-1].word
    st2 = stack1[-2].word
    left_st1 = find_left(st1,dependencies,left_list)
    left_st2 = find_left(st2,dependencies,left_list)
    right_st1 = find_right(st1,dependencies,right_list)
    right_st2 = find_right(st2,dependencies,right_list)
    ans = [left_st1,left_st2,right_st1,right_st2]
    return ans






In [124]:
null_token = Token(-2,'NULL','NULL')
root_taken = Token(-1,'ROOT', 'NULL')
pos_dict ={}
tags_dict = {}
with open("tagset.txt") as fic:
    i = 0
    for line in fic:
      tag = line.rstrip('\n')
      tags_dict [tag] = i
      i = i+1
# print(tags_dict)
with open("pos_set.txt") as fic:
    i = 0
    for line in fic:
      tag = line.rstrip('\n')
      pos_dict [tag] = i
      i = i+1
# print(pos_dict)
rev_tags_dict = {}
with open("tagset.txt") as fic:
    i = 0
    for line in fic:
      tag = line.rstrip('\n')
      rev_tags_dict [i] = tag
      i = i+1
# print(rev_tags_dict)


In [125]:
train_data =[]
label_data = []

for i in range(len(string_arr)):
  # print('example',i)
  stack = [null_token,root_taken]
  parse_buffer = []
  dependencies =[]

  for j in range(len(string_arr[i])):
    parse_buffer.append(Token(j+1,string_arr[i][j],pos_arr[i][j]))
  parse_buffer.append(Token(j+2,'NULL','NULL'))
  parse_buffer.append(Token(j+3,'NULL','NULL'))

  # temp_state = ParseState(stack,parse_buffer,dependencies)
  k = ParseState(stack,parse_buffer,dependencies)
  j = 0

  while(not is_final_state(k,2)):
    # print(j)
    deps = left_right(k)
    train_example = []
    train_example.append(stack[-1].word)
    train_example.append(stack[-2].word)
    train_example.append(parse_buffer[0].word)
    train_example.append(parse_buffer[1].word)
    train_example.append(deps[0])
    train_example.append(deps[1])
    train_example.append(deps[2])
    train_example.append(deps[3])
    train_example.append(pos_dict[stack[-1].pos])
    train_example.append(pos_dict[stack[-2].pos])
    train_example.append(pos_dict[parse_buffer[0].pos])
    train_example.append(pos_dict[parse_buffer[1].pos])

    train_data.append(train_example)
    action = actions_arr[i][j]

    label_data.append(tags_dict[action])
    j = j+1

    if(tags_dict[action]!= 0):
      if(tags_dict[action]) % 2 == 1:
        left_arc(k,action)
      else:
        right_arc(k,action)
    else:
      shift(k)





In [126]:
print(train_data[1])

['Aesthetic', 'ROOT', 'Appreciation', 'and', 'NULL', 'NULL', 'NULL', 'NULL', 0, 17, 3, 8]


In [127]:
import numpy as np
final_train_data =[]
lenth = 50
for i in range(len(train_data)):
  # print(i)
  temp =np.zeros(lenth).tolist()
  temp1 =np.zeros(lenth).tolist()
  for j in range(12):
    if(j<4):
      vector = vec[train_data[i][j].lower()].tolist()
      for k in range(lenth):
        temp[k] = temp[k] + vector[k]
    elif(j<8):
      vector = vec[train_data[i][j].lower()].tolist()
      for k in range(lenth):
        temp1[k] = temp1[k] + vector[k]
      if(j == 7):
        temp.extend(temp1)
    else:
      # print(temp)
      temp.append(train_data[i][j])
  final_train_data.append(temp.copy())




In [128]:
print(len(final_train_data[0]))

104


In [129]:
dev_string_arr=[]
dev_pos_arr=[]
dev_actions_arr=[]
with open("dev.txt") as fic:
    for line in fic:
      dev_lines = line.rstrip('\n').split('|||')
      dev_line0 = dev_lines[0].split(' ')
      dev_line1 = dev_lines[1].split(' ')
      dev_line2 = dev_lines[2].split(' ')
      dev_string_arr.append(dev_line0[0:-1])
      dev_pos_arr.append(dev_line1[1:-1])
      dev_actions_arr.append(dev_line2[1:])

In [130]:
import torch
class MLP(torch.nn.Module):
  def __init__(self,d_pos, embed_dim,word_embed_dim, hid_dim,out_dim):
    """ This is the constructor. Initialize your network
    layers here.
    Inputs
    --------
    in_dim: int. Dimension of your input. It was 2 in the example
    hid_dim: int. Dimension of the intermediate level. Also 2 in the example
    out_dim: int. Dimension of your output. It was 1 in the example
    """
    super().__init__()
    self.embed = torch.nn.Embedding(d_pos, embed_dim)        # Invoke the constructor of the parent class
    self.l1 = torch.nn.Linear(word_embed_dim, hid_dim,bias = True)
    self.l2 = torch.nn.Linear(embed_dim, hid_dim, bias = True)
    self.lnew = torch.nn.Linear(word_embed_dim, hid_dim, bias = True)
    self.re = torch.nn.ReLU()
    self.l3 = torch.nn.Linear(hid_dim,out_dim,bias = True)

  def forward(self, inp):
    """ Method for forward pass
    Input
    -------
    inp: torch.Tensor. The dimension of the input (i.e, the last number in shape)
        should match `in_dim`. The shape should be (n,in_dim) where n is the batch size

    Output
    ------
    z: torch.Tensor. The result of the forward pass. The dimension of the output
     (i.e, the last number in shape) should match `out_dim`. The shape should be
     (n, out_dim) where n is the batch size.
    """
    a = inp[:,:50]
    b = inp[:,50:100]
    c = inp[:,100:].to(torch.int)
    z = self.l3(self.re(torch.add(torch.add(self.l2(torch.sum(self.embed(c),dim=1)),self.l1(a)),self.lnew(b))))
    return z

device = torch.device(f"cuda" if torch.cuda.is_available() else "cpu")
model = MLP(d_pos = 18, embed_dim=50, word_embed_dim = lenth,hid_dim=200,out_dim=75).to(device)

In [131]:
train_dataset = TensorDataset(
                          torch.tensor(final_train_data),
                          torch.tensor(label_data))
# dev_dataset = TensorDataset(
#                           torch.tensor(dev_list, dtype = torch.int),
#                           torch.tensor(dev_label_list))

In [132]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [133]:
loss_fn = torch.nn.CrossEntropyLoss()

In [134]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [135]:
from tqdm import tqdm   # This library allows us to see how far in the epoch we are
import numpy as np
import time
from sklearn.metrics import f1_score

# We need to decide what device we'll be using for our model computations
# The following like says that if any GPU is available, then use it.
# Otherwise, we use a CPU
# device = torch.device(f"cuda" if torch.cuda.is_available() else "cpu")

max_epochs = 20

best_perf_dict = {"metric": 0, "epoch": 0}

# Create a randomly initialized model and the optimizer.
# model = MLP(in_dim=16, hid_dim=12, out_dim=7).to(device)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

# Begin the training loop
for ep in range(1, max_epochs+1):
    print(f"\nEpoch {ep}")
    train_loss = []       # We will use this list to accumulate losses and observe if training is happening

    # We load data in batches from the loader and will do backpropagation
    # for this batch of inputs in every iteration
    for inp, lab in tqdm(train_loader):
        # Switch to train mode
        # This doesn't make a difference to our model but is considered good practice
        # Models might contain layers like Dropout which behave differently when training
        # versus evaluating. This signals these layers to behave appropriately.
        model.train()

        # PyTorch accumulates gradients for every `.backward()`
        # This means if we do not set it to zero, it will add up all the
        # gradients it has seen till that moment.
        optimizer.zero_grad()

        # Let us check what the input and the label shapes are
        # print(f"\nInput shape: {inp.shape}")
        # print(f"Label shape: {lab.shape}")

        out = model(inp.to(device))    #Forward pass
        # print(f"Output shape: {out.shape}")  # Output shape

        # We have the output logits and the labels. We can compute the loss now
        loss = loss_fn(out, lab.to(device))

        # print(f"Loss: {loss} with shape {loss.shape}")

        loss.backward() # computing the gradients
        optimizer.step()  # Performs the optimization

        train_loss.append(loss.cpu().item())    # Appending the batch loss to the list

    print(f"Average training batch loss: {np.mean(train_loss)}\n")

    # train_data =[]
    # label_data = []
    model.eval()
    dev_actions_pred_arr=[]
    for i in range(len(dev_string_arr)):
      temp_actions_pred_arr = []
      # print('example',i)
      dev_stack = [null_token,root_taken]
      dev_parse_buffer = []
      dev_dependencies =[]

      for j in range(len(dev_string_arr[i])):
        dev_parse_buffer.append(Token(j+1,dev_string_arr[i][j],dev_pos_arr[i][j]))
      dev_parse_buffer.append(Token(j+2,'NULL','NULL'))
      dev_parse_buffer.append(Token(j+3,'NULL','NULL'))

      # temp_state = ParseState(stack,parse_buffer,dependencies)
      dev_k = ParseState(dev_stack,dev_parse_buffer,dev_dependencies)
      j = 0

      while(not is_final_state(dev_k,2)):
        # print(j)
        deps = left_right(dev_k)
        train_example = []
        train_example.append(dev_stack[-1].word)
        train_example.append(dev_stack[-2].word)
        train_example.append(dev_parse_buffer[0].word)
        train_example.append(dev_parse_buffer[1].word)
        train_example.append(deps[0])
        train_example.append(deps[1])
        train_example.append(deps[2])
        train_example.append(deps[3])
        train_example.append(pos_dict[dev_stack[-1].pos])
        train_example.append(pos_dict[dev_stack[-2].pos])
        train_example.append(pos_dict[dev_parse_buffer[0].pos])
        train_example.append(pos_dict[dev_parse_buffer[1].pos])
        # train_data.append(train_example)

        temp =np.zeros(lenth).tolist()
        temp1 =np.zeros(lenth).tolist()
        for p in range(12):
          if(p<4):
            vector = vec[train_example[p].lower()].tolist()
            for k in range(lenth):
              temp[k] = temp[k] + vector[k]
          elif(p<8):
            vector = vec[train_example[p].lower()].tolist()
            for k in range(lenth):
              temp1[k] = temp1[k] + vector[k]
            if(p == 7):
              temp.extend(temp1)
          else:
            # print(temp)
            temp.append(train_example[p])
        input = []
        input.append(temp.copy())
        with torch.no_grad():
          out = model(torch.tensor(input).to(device))
        out1 = out[0].cpu().tolist()
        action = find_action(dev_k,out1)#actions_arr[i][j]
        temp_actions_pred_arr.append(action)
        j = j+1
        # label_data.append(tags_dict[action])

        if(tags_dict[action]!= 0):
          if(tags_dict[action]) % 2 == 1:
            left_arc(dev_k,action)
          else:
            right_arc(dev_k,action)
        else:
          shift(dev_k)
      # print(temp_actions_pred_arr)
      dev_actions_pred_arr.append(temp_actions_pred_arr.copy())

    dev_accuracy = compute_metrics(dev_string_arr,dev_actions_arr,dev_actions_pred_arr,2)
    print(f"(UAS,LAS): {dev_accuracy}\n")
    (accuracy_uas,accuracy_las) = dev_accuracy
    if accuracy_las > best_perf_dict["metric"]:
        best_perf_dict["metric"] = accuracy_las
        best_perf_dict["epoch"]  = ep
        torch.save({
            "model_param": model.state_dict(),
            "optim_param": optimizer.state_dict(),
            "dev_metric": accuracy_las,
            "epoch": ep
        }, f"./models/{ep}")




Epoch 1


100%|██████████| 4024/4024 [00:14<00:00, 274.62it/s]


Average training batch loss: 1.1946034585357186

(UAS,LAS): (0.3986300600752344, 0.33035764415249)


Epoch 2


100%|██████████| 4024/4024 [00:14<00:00, 272.21it/s]


Average training batch loss: 1.089858064846419

(UAS,LAS): (0.4168210656335972, 0.35326483633709505)


Epoch 3


...

Average training batch loss: 0.9940250064163038

(UAS,LAS): (0.4375947448206165, 0.374487676155185)


Epoch 19


100%|██████████| 4024/4024 [00:19<00:00, 201.34it/s]


Average training batch loss: 0.9927369102569034

(UAS,LAS): (0.4432653977878839, 0.382011116725619)


Epoch 20


100%|██████████| 4024/4024 [00:20<00:00, 197.44it/s]


Average training batch loss: 0.9924691074174157

(UAS,LAS): (0.4327662680366066, 0.3774633653360283)



In [136]:
best_epoch = best_perf_dict["epoch"]
model_path = f"./models/{best_epoch}"
checkpoint = torch.load(model_path)

model.load_state_dict(checkpoint["model_param"])
optimizer.load_state_dict(checkpoint["optim_param"])

print(f"""Dev LAS of best model: {checkpoint["dev_metric"]} at epoch {checkpoint["epoch"]}""")


Dev LAS of best model: 0.39329627758127 at epoch 6


In [137]:
test_string_arr=[]
test_pos_arr=[]
test_actions_arr=[]
with open("test.txt") as fic:
    for line in fic:
      test_lines = line.rstrip('\n').split('|||')
      test_line0 = test_lines[0].split(' ')
      test_line1 = test_lines[1].split(' ')
      test_line2 = test_lines[2].split(' ')
      test_string_arr.append(test_line0[0:-1])
      test_pos_arr.append(test_line1[1:-1])
      test_actions_arr.append(test_line2[1:])

In [138]:
print(test_actions_arr[0:5])

[['SHIFT', 'SHIFT', 'REDUCE_L_det', 'SHIFT', 'SHIFT', 'REDUCE_L_case', 'REDUCE_R_nmod', 'SHIFT', 'SHIFT', 'SHIFT', 'REDUCE_L_amod', 'REDUCE_L_case', 'REDUCE_R_nmod', 'SHIFT', 'SHIFT', 'SHIFT', 'REDUCE_L_amod', 'REDUCE_L_case', 'REDUCE_R_nmod', 'SHIFT', 'REDUCE_R_punct'], ['SHIFT', 'SHIFT', 'SHIFT', 'SHIFT', 'SHIFT', 'REDUCE_L_advmod', 'SHIFT', 'REDUCE_L_amod', 'REDUCE_L_det', 'REDUCE_L_case', 'SHIFT', 'SHIFT', 'REDUCE_L_case', 'REDUCE_R_nmod', 'REDUCE_R_nmod'], ['SHIFT', 'SHIFT', 'REDUCE_R_punct'], ['SHIFT', 'SHIFT', 'REDUCE_L_amod', 'SHIFT', 'SHIFT', 'REDUCE_L_case', 'SHIFT', 'SHIFT', 'REDUCE_L_cc', 'REDUCE_R_conj', 'REDUCE_R_nmod', 'SHIFT', 'SHIFT', 'SHIFT', 'SHIFT', 'REDUCE_L_det', 'REDUCE_L_cop', 'REDUCE_L_aux', 'REDUCE_L_nsubj', 'SHIFT', 'SHIFT', 'SHIFT', 'SHIFT', 'REDUCE_L_amod', 'SHIFT', 'REDUCE_L_compound', 'REDUCE_L_dep', 'REDUCE_L_case', 'REDUCE_R_nmod', 'SHIFT', 'REDUCE_R_punct', 'SHIFT', 'SHIFT', 'REDUCE_L_punct', 'SHIFT', 'SHIFT', 'REDUCE_L_case', 'REDUCE_R_nmod', 'SHIFT',

In [139]:
    model.eval()
    test_actions_pred_arr=[]
    for i in range(len(test_string_arr)):
      temp_actions_pred_arr = []
      # print('example',i)
      test_stack = [null_token,root_taken]
      test_parse_buffer = []
      test_dependencies =[]

      for j in range(len(test_string_arr[i])):
        test_parse_buffer.append(Token(j+1,test_string_arr[i][j],test_pos_arr[i][j]))
      test_parse_buffer.append(Token(j+2,'NULL','NULL'))
      test_parse_buffer.append(Token(j+3,'NULL','NULL'))

      # temp_state = ParseState(stack,parse_buffer,dependencies)
      dev_k = ParseState(test_stack,test_parse_buffer,test_dependencies)
      j = 0

      while(not is_final_state(dev_k,2)):
        # print(j)
        deps = left_right(dev_k)
        train_example = []
        train_example.append(test_stack[-1].word)
        train_example.append(test_stack[-2].word)
        train_example.append(test_parse_buffer[0].word)
        train_example.append(test_parse_buffer[1].word)
        train_example.append(deps[0])
        train_example.append(deps[1])
        train_example.append(deps[2])
        train_example.append(deps[3])
        train_example.append(pos_dict[test_stack[-1].pos])
        train_example.append(pos_dict[test_stack[-2].pos])
        train_example.append(pos_dict[test_parse_buffer[0].pos])
        train_example.append(pos_dict[test_parse_buffer[1].pos])
        # train_data.append(train_example)

        temp =np.zeros(lenth).tolist()
        temp1 =np.zeros(lenth).tolist()
        for p in range(12):
          if(p<4):
            vector = vec[train_example[p].lower()].tolist()
            for k in range(lenth):
              temp[k] = temp[k] + vector[k]
          elif(p<8):
            vector = vec[train_example[p].lower()].tolist()
            for k in range(lenth):
              temp1[k] = temp1[k] + vector[k]
            if(p == 7):
              temp.extend(temp1)
          else:
            # print(temp)
            temp.append(train_example[p])
        input = []
        input.append(temp.copy())
        with torch.no_grad():
          out = model(torch.tensor(input).to(device))
        out1 = out[0].cpu().tolist()
        action = find_action(dev_k,out1)#actions_arr[i][j]
        temp_actions_pred_arr.append(action)
        j = j+1
        # label_data.append(tags_dict[action])

        if(tags_dict[action]!= 0):
          if(tags_dict[action]) % 2 == 1:
            left_arc(dev_k,action)
          else:
            right_arc(dev_k,action)
        else:
          shift(dev_k)
      # print(temp_actions_pred_arr)
      test_actions_pred_arr.append(temp_actions_pred_arr.copy())

    test_accuracy = compute_metrics(test_string_arr,test_actions_arr,test_actions_pred_arr,2)

In [ ]:
(a,b) =test_accuracy
print(f"""Test LAS of best model:  {b}""")